In [864]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np 
import random

In [865]:
class Random_Variable: 
    def __init__(self, name, values, probability_distribution): 
        self.name = name 
        self.values = values 
        self.probability_distribution = probability_distribution 
        if all(type(item) is np.int64 for item in values): 
            self.type = 'numeric'
            self.rv = stats.rv_discrete(name = name, values = (values, probability_distribution))
        elif all(type(item) is str for item in values): 
            self.type = 'symbolic'
            self.rv = stats.rv_discrete(name = name, values = (np.arange(len(values)), probability_distribution))
            self.symbolic_values = values 
        else: 
            self.type = 'undefined'
            
    def sample(self,size): 
        if (self.type =='numeric'): 
            return self.rv.rvs(size=size)
        elif (self.type == 'symbolic'): 
            numeric_samples = self.rv.rvs(size=size)
            mapped_samples = [self.values[x] for x in numeric_samples]
            return mapped_samples 

In [866]:
def diceRoll(sides):
    return random.randrange(1,sides+1)

def d20():
    return random.randrange(1,21)

In [867]:
def strikeHit(targetAC, hitResult):
    # regular hit
    if(hitResult>=targetAC+10):
        return 2
    elif(hitResult>=targetAC):
        return 1
    else:
        return 0

In [868]:
# Test 

attackDamage = 5 + 2*diceRoll(8)


# Can the PC survive focus fire?
pcHP = 156
pcAC = 27

def EhyreenAttack(pcAC,MAP):
    return strikeHit(pcAC,25-MAP)* (2*diceRoll(8) + 13)

def EhyreenAttack3xDMG(pcAC):
    damage = 0
    damage += EhyreenAttack(pcAC,0)
    damage += EhyreenAttack(pcAC,5)
    damage += EhyreenAttack(pcAC,10)
    return damage




In [869]:
# get did hit? damge, pipe that into, handle
# PC class specific implementation, so, can reduce with shield. etc

In [870]:
# samples = []
# died = 0

# trials = 10000
# for i in range(1,10000):
#     if(pcHP<EhyreenAttack3xDMG(pcAC)):
#         died += 1

# died = died/trials

# print(died)


In [871]:
# Inside methods, we can do all kinds special defensive things/logic
class pcOrc:
    def __init__(self):
        self.actions = 3
        self.reactions = 1
        self.hpMax = 157
        self.hp = self.hpMax
        self.AC = 27

        self.OrcFerocityTriggered = 0
    
    def turnStart():
        self.actions = 3
        self.reactions = 1

    def CheckHit(self,attackRoll):
        # Can implement on demand extra AC features here
        return strikeHit(self.AC,attackRoll)
    
    # Ideally damage would be a struct, with different kinds of damage assigned.
    def TakesDamage(self,damage):
        #Orc Ferocity https://2e.aonprd.com/Feats.aspx?ID=4514
        if(self.reactions and self.hp-damage<=0 and 2*damage < self.hpMax):
            self.reactions = 0
            self.hp = 1
            self.OrcFerocityTriggered = 1
            return
        
        self.hp -= damage

        

In [872]:
# Sim does

# 1 is hit, 2 crit, 0 miss
# Make 3 attacks
def Eriny3xAttacks(target):
    mapArray = [0,-5,-10]
    for curMap in mapArray:
        ErinyHitResult = target.CheckHit(d20() + 25 - curMap)
        if(ErinyHitResult>0):
            # compute damage
            damage = 2*diceRoll(8) + 6 + diceRoll(6) + diceRoll(6)

            # factor in crit bonus
            damage *= ErinyHitResult
            
            target.TakesDamage(damage)

In [873]:
def Hamatula3xAttacks(target):
    mapArray = [0,-5,-10]
    for curMap in mapArray:
        HitResult = target.CheckHit(d20() + 24 - curMap)
        if(HitResult>0):
            # compute damage
            damage = 3*diceRoll(8) + 13 + diceRoll(6)

            # factor in crit bonus
            damage *= HitResult
            
            target.TakesDamage(damage)

In [874]:
# PC
# Does PC get 1 turned

samples = []

died = 0
OrcFerocityProccs = 0

trials = 100000
for i in range(1,trials):
    testPC = pcOrc()
    Hamatula3xAttacks(testPC)
    died += testPC.hp<=0
    OrcFerocityProccs += testPC.OrcFerocityTriggered

print(died/trials)
print(OrcFerocityProccs/trials)

0.36988
0.0
